# Pokemon Chatbot using NLTK
---

My goal for this project is to create a very simple chatbot that uses NLTK and the PyPokedex libraries to give the user information about a requested pokemon.
The name of the bot is **PokeBot**. It is a rule/keyword based chatbot with limited conversation about Pokemon.

### PokeBot Can:
-[x] Greet the user
-[x] Respond to thank you messages
-[x] Close if the user says goodbye
-[x] Provide information about a pokemon such as:
> ###### Name 

> ###### Type

> ###### Height

> ###### Weight

> ###### Base Experience

> ###### Abilities

> ###### Basic Stats

---


# Action Plan
---

### HOW I CREATED THE BOT


##### Download and Install Packages

In [ ]:
%pip install nltk
%pip install pandas
%pip install pypokedex

#### Create the Rules for the Chatbot from this .json File.

```json
{
    "utterance_response": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi",
                "Hey",
                "Hello",
                "Hey there",
                "Howdy",
                "Sup",
                "Wassup",
                "What's up",
                "What's good"
            ],
            "responses": [
                "Hey there!",
                "Hi!",
                "Hello!",
                "Hello, thanks for stopping by"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "Goodbye",
                "See you later",
                "See you",
                "See ya",
                "Catch ya",
                "Catch you later",
                "Quit",
                "Quit talking",
                "Quit talking to me"
            ],
            "responses": [
                "Bye!",
                "Goodbye!",
                "See you later!",
                "Bye take care. See you soon :) ",
                "It was nice talking to you. See you soon :)"
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "Thank you so much",
                "That's very kind of you",
                "That's helpful",
                "That's very helpful"
            ],
            "responses": [
                "You're welcome!",
                "No problem!",
                "You're welcome!",
                "Any time!",
                "My pleasure!"
            ]
        },
        {
            "tag": "name",
            "patterns": [
                "My name is",
                "I am",
                "I'm",
                "I am called",
                "I'm called"
            ],
            "responses": [
                "Nice to meet you",
                "Hi"
            ],
            "context_set": "name"
        },
        {
            "tag": "pokemon",
            "patterns": [
                "which pokemon is ",
                "what is ",
                "tell me about ",
                "i want to know about ",
                "i want to know more about "
            ],
            "responses": [
                "Here's what I know",
                "Alright, here's what I know",
                "Sure",
                "No problem! Let me get that for you"
            ],
            "context_set": "pokemon"
        }
    ]
}

```

#### Save Rules in a Useable Format

In [51]:
import json
import re
import pickle as pkl
from preprocess import normalize

In [52]:
# load the json file
with open('utterance_response.json') as file:
    data = json.load(file)

# function used to create rules data structure
def save():
    intents = {}
    keywords_dict = {}
    responses_dict = {}

    for intent in data['utterance_response']:
        # save the intent tag, which is either 'greeting' or 'goodbye' or 'thanks' or 'name' or 'pokemon'
        tag = intent['tag'] 
        intents[tag] = []
        # build the dictionary of responses for each intent
        responses_dict[tag] = intent['responses']
        # normalize the patterns associated with each intent
        for pattern in intent['patterns']:
            # use regex to format the patterns so they can be found with regex search later
            pattern = [f'.*\\b{pat}\\b.*' for pat in normalize(pattern)]
            intents[tag].extend([pat for pat in pattern if pat not in intents[tag]])
    #build the dictonary of intents for each pattern
    for intent, keys in intents.items():
        keywords_dict[intent] = re.compile('|'.join(keys))

    # save dictionary of patterns and responses in 2 separate pickle files
    utterance_patterns = open("C:/Users/Swan/Documents/Advanced_Natural_Language_Processing/ClassProject/utterance_patterns.pkl", "wb")
    response_patterns = open("C:/Users/Swan/Documents/Advanced_Natural_Language_Processing/ClassProject/response_patterns.pkl", "wb")
    pkl.dump(keywords_dict, utterance_patterns)
    pkl.dump(responses_dict, response_patterns)

### POKEBOT ALGORITHM

In [53]:
import random
import pandas as pd
import pickle as pkl
import pypokedex as pdx

#### Get Utterance and Response Dictionaries that were Previously Created

In [54]:
# Load data intents data
with open('C:/Users/Swan/Documents/Advanced_Natural_Language_Processing/ClassProject/utterance_patterns.pkl', 'rb') as utt_pkl:
    utterances = pkl.load(utt_pkl)
with open('C:/Users/Swan/Documents/Advanced_Natural_Language_Processing/ClassProject/response_patterns.pkl', 'rb') as resp_pkl:
    responses = pkl.load(resp_pkl)

In [55]:
# Get pokemon names from csv file found at https://gist.github.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6
pokemon_df = pd.read_csv('pokemon.csv')
pokemon_names = [name.lower() for name in pokemon_df['NAME']]

# name of bot
bot_name = "PokeBot"

#### Use PyPokeDex to Obtain Pokemon Information

In [56]:
def get_pokemon(pokemon_name):
    try:
        pokemon = pdx.get(name=pokemon_name)
        response = f"Meet {pokemon.name.title()}!" \
                   f"\n{pokemon.name.title()} has these attributes:" \
                   f"\nType: {pokemon.types[0].title()}!" \
                   f"\nHeight: {pokemon.height}!" \
                   f"\nWeight is {pokemon.weight}!" \
                   f"\nBase Experience is {pokemon.base_experience}!" \
                   f"\nAbilities are {pokemon.abilities[0].name, pokemon.abilities[1].name}!" \
                   f"\nStats are hp={pokemon.base_stats[0]}, attack={pokemon.base_stats[1]}, defense" \
                   f"={pokemon.base_stats[2]}, sp_atk={pokemon.base_stats[3]}, sp_def={pokemon.base_stats[4]}," \
                   f"speed={pokemon.base_stats[5]}"
    except:
        response = "\tSorry, I don't know that Pokemon."
    return response


#### Get User Input and Return Appropriate Response

In [69]:
def get_response(user_input):
    """This function takes in a user's input, normalizes it, identifies the user's intent and returns a response"""
    pokemon_name = ""
    user_input = normalize(user_input)
    matched_intent = None

    for intent, pattern in utterances.items():
        for word in user_input:
            # if the word matches a pattern in the dictionary, set the matched_intent to the intent
            if re.search(pattern, word):
                matched_intent = intent
                break

    # if the matched_intent is None, then the user's input was not recognized as an intent and may be a pokemon name
    if (matched_intent is None) or (matched_intent == 'pokemon'):
        for word in user_input:
            # if the word is a pokemon name/id, set the pokemon_name to the word and set the matched_intent to pokemon
            if word in pokemon_names or word.isnumeric():
                pokemon_name = word
                matched_intent = 'pokemon'
                break

    # if the matched_intent is in the dictionary, then the user's input was recognized as an intent
    # use the matched_intent to get a response from the dictionary
    if matched_intent in responses:
        key = matched_intent

        # if the matched_intent is good bye, then the bot will exit the conversation
        if key == 'goodbye':
            exit(random.choice(responses[key]))          

        # if the matched_intent is a person's name, then the bot will greet the user
        if key == 'name':
            return f'{random.choice(responses[key])} {user_input[1].title()}!'
        # else if the mathced_intent is pokemon, then the bot will return the pokemon's information
        elif key == 'pokemon':
            pokemon_details = get_pokemon(pokemon_name)
            return f"{random.choice(responses[key])}\n{pokemon_details}"
        else:
            return random.choice(responses[key])
    # if the matched_intent is not in the dictionary, then the user's input was not recognized as an intent
    else:
        return "Sorry, I don't understand"

#### Repeat Algorithm until Response is to EXIT/QUIT

In [60]:
def main():
    print(f"Hi! I'm {bot_name}!")
    print("Let's talk about your Pokemon!")
    while True:
        user_input = input("\nYou: ")
        response = get_response(user_input)
        print(f"\n{bot_name}: {response}")

### ALGORITHM USED TO NORMALIZE INPUT


**Replace Contractions**
```py
    # replace contractions with full words
    expand_contractions(sentence).lower()
```


**Remove Non-Ascii Characters**

```py
    # remove non-ascii characters
    sentence = unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore')
```

**Remove Punctuations**
```py
    # remove punctuation
    re.sub(r'[^\w\s]', '', sentence)
```

**Tokenize**
```py
    # tokenize into words
    nltk.word_tokenize(sentence)
```

**Remove StopWords**
```py
    #remove stopwords
    [word for word in sentence if word not in nltk.corpus.stopwords.words('english')]
```

**Lemmatize**
```py
    #lemmatize
    [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in sentence]
```

**Stemming**
```py
    #stemm
    [nltk.stem.PorterStemmer().stem(word) for word in sentence]
```

**Remove Single Characters that are NOT numerical**
```py
    # remove single characters if any were missed
    # keep numerical values because they can be used to find a pokemon
    [word for word in sentence if (len(word) > 1 or word.isnumeric())]
```

**_Next the word is returned_**

### User Interface
---

For the user interface of this bot. I decided to use Python's TKinter Library.

In [ ]:
import tkinter as tk


"""UI for the PokeBot credits to Python Engineer @ https://www.youtube.com/watch?v=RNEcewpVZUQ"""
BG_GRAY_COLOR = '#37796C'
BG_COLOR = '#0C3348'
SEND_BUTTON_COLOR = '#E54222'
TEXT_COLOR = '#EAECEE'
FONT = "Helvetica 10"
FONT_BOLD = "Arial 14 bold"


class PokeBot(tk.Tk):
    def __init__(self):
        self.window = tk.Tk()
        self._setup_main_window()

    def _setup_main_window(self):
        self.window.title("Chat with PokeBot")
        self.window.resizable(False, False)
        self.window.configure(width=570, height=650, bg=BG_GRAY_COLOR)
        self.window.config(padx=10, pady=10)

        # TITLE
        title_label = tk.Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR, text="PokeBot",
                               font=FONT_BOLD, padx=10, pady=10)
        title_label.place(relwidth=1, relheight=0.1)

        # DIVIDER
        divider = tk.Label(self.window, width=450, bg=BG_GRAY_COLOR, height=2)
        divider.place(relwidth=1, relheight=0.01, relx=0, rely=0.07)

        # TEXT INPUT
        self.text_widget = tk.Text(self.window, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT,
                                   height=2, width=20, padx=5, pady=5)
        self.text_widget.place(relwidth=1, relheight=0.745, relx=0, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=tk.DISABLED)

        # SCROLLBAR
        scrollbar = tk.Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # BOTTOM LABEL
        bottom_label = tk.Label(self.window, bg=BG_GRAY_COLOR, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # MESSAGE BOX
        self.msg_entry = tk.Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter)

        # SEND BUTTON
        send_button = tk.Button(bottom_label, text="Send", bg=SEND_BUTTON_COLOR, fg=TEXT_COLOR, font=FONT_BOLD,
                                width=20, command=lambda: self._on_enter(self.msg_entry.get()))
        send_button.place(relwidth=0.22, relheight=0.06, rely=0.008, relx=0.77)


    # EVENT HANDLERS
    # fires when the enter key is pressed
    def _on_enter(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You: ")

    # inserts a message into the text widget
    # happens when the _on_enter event fires
    def _insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, tk.END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=tk.NORMAL)
        self.text_widget.insert(tk.END, msg1)
        self.text_widget.configure(state=tk.DISABLED)

        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state=tk.NORMAL)
        self.text_widget.insert(tk.END, msg2)
        self.text_widget.configure(state=tk.DISABLED)

        self.text_widget.see(tk.END)

    def run(self):
        self.window.mainloop()

# Code in Action
___

In [78]:
if __name__ == "__main__":
    bot = PokeBot()
    bot.run()